<a href="https://colab.research.google.com/github/Jinukki/KJU/blob/master/A2C_ch_4_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ch.4-4 $\mid$ A2C 알고리즘
---

### 샘플링 기법으로 근사적으로 계산한 목적함수 그래디언트 식과, 한 개의 에피소드만 고려한 목적함수 그래디언트는 다음과 같다.(m은 에피소드 인덱스, M은 에피소드 개수)<br>
$$\nabla_\theta J(\theta)\approx \sum_{t=0}^T[\frac{1}{M}\sum_{m=1}^M(\nabla_\theta log\pi_\theta(u_{t}^{(m)}\mid x_{t}^{(m)})A^{\pi_\theta}(x_{t}^{(m)}, u_{t}^{(m)}))]$$
<br>
$$\nabla_\theta J(\theta)\approx \sum_{t=0}^T(\nabla_\theta log\pi_\theta(u_{t}^{(m)}\mid x_{t}^{(m)})A^{\pi_\theta}(x_{t}^{(m)}, u_{t}^{(m)}))$$
<br>

### 위 식에서 어드밴티지 함수는 행동가치 함수와 상태가치 함수의 차이며, 행동가치 함수 식은 아래와 같이 나타낼 수 있으므로, 어드밴티지 계산의 문제를 상태가치를 얼마나 정확히 계산하느냐의 문제로 나타낼 수 있다.<br>
$$Q^\pi(x_t, u_t)=r(x_t, u_t)+E_{x_{t+1}\sim p(x_{t+1}\mid x_t, u_t)}[\gamma V^{\pi}(x_{t+1})]$$
<br>
$$A^{\pi_\theta}(x_t, u_t)=Q^{\pi_\theta}(x_t, u_t)-V^{\pi_\theta}(x_{t}) \approx r(x_t, u_t)+\gamma V^{\pi_\theta}(x_{t+1})-V^{\pi_\theta}(x_{t})$$
<br>

### 정책 신경망을 이용해 정책을 근사했듯이 다른 신경망을 통해 상태가치 함수를 근사한다. 정책 신경망은 $\pi_\theta(u_t\mid x_t)$ 를 추정하고 가치 신경망은 $V_\phi(x_t)\approx V^{\pi_\theta}(x_{t})$ 를 추정한다
<br>

### 여기서 정책 신경망은 에이전트가 어떻게 행동해야 하는지를 알려주므로 <b>액터(actor) 신경망</b>라고 하고, 가치 신경망은 그 행동을 평가하기 때문에 <b>크리틱(critic) 신경망</b>이라고 한다. 각 신경망에 대한 그림과, 두 신경망의 중첩을 고려하여 합쳐서 나타낸 그림은 아래와 같다.

![1](https://user-images.githubusercontent.com/52277776/81943418-51067700-9636-11ea-9387-57c2d067daf7.jpg)

<br>

![2](https://user-images.githubusercontent.com/52277776/81943420-52d03a80-9636-11ea-9d36-f514b3dbe2ef.jpg)


### 가치함수를 근사하는 함수 $V_\phi(x_t)\approx V^{\pi_\theta}(x_{t})$를 구하기 위해 벨만 방정식을 이용하면 가치함수를 근사하는 함수는 다음과 같이 추정 가능하다.<br>
$$V_\phi(x_t)\approx r(x_t, u_t)+\gamma V_\phi(x_{t+1})$$
<br>

### 시간차 타깃(TD target)을 $y_i=r(x_t, u_t)+\gamma V_\phi(x_{t+1})$로 설정하면 다음 손실함수가 최소가 되도록 근사 가치함수 $V_\phi(x_t)$를 추정 가능하다. <br>
$$Loss_{critic}(\phi)=\frac{1}{2}\sum_{i}\mid\mid r(x_i, u_i)+\gamma V_\phi(x_{i+1})-V_\phi(x_i)\mid\mid^2$$
<br>

### 어드밴티지도 다음과 같이 근사할 수 있다.<br>
$$A_\phi(x_i, u_i)\approx r(x_t, u_t)+\gamma V_\phi(x_{i+1})-V_\phi(x_i)$$
<br>
### 또한,<br>
$$\theta\leftarrow\theta-\alpha\nabla_\theta\sum_{i}(log\pi_\theta(u_i\mid x_i)A_\phi(x_i, u_i))$$
<br>

### 이기 때문에, 액터 신경망의 손실함수는 다음과 간다.<br>
$$Loss_{actor}(\theta)\approx -\sum_{i}(log\pi_\theta(u_i\mid x_i)A_\phi(x_i, u_i))$$

### 아래 그림은 어드밴티치 액터-크리틱(A2C) 알고리즘의 구조를 보여준다.

![3](https://user-images.githubusercontent.com/52277776/81943430-55329480-9636-11ea-9b0a-fce1ad8ef61c.jpg)

### 배치(batch) 형태로 나타내면 아래와 같고, 정책 $\pi_{\theta_i}$을 N 시간스텝 동안 실행시켜 N 개의 샘플을 생성하고 폐기해가며 정책을 업데이트 한다. 이 과정을 일정 성능에 도달할 때까지 반복하며 학습을 진행한다.

![4](https://user-images.githubusercontent.com/52277776/81943434-5794ee80-9636-11ea-85c4-7f60dec9a4be.jpg)

### 한 개의 샘플이 생성되는 즉시 신경망을 업데이트 하는 온라인 액터-크리틱 신경망의 알고리즘은 아래와 같다.

![5](https://user-images.githubusercontent.com/52277776/81943447-5a8fdf00-9636-11ea-9961-1ecd7f9842bd.jpg)


### 연속공간 행동변수일 경우 액터 신경망의 출력인 정책은 임의의 확률밀도함수이므로, 이를 표현하기 위해 무한 개의 뉴런이 필요하다.

### 이 문제를 해결하기 위해 행동변수가 서로 독립인 가우시안이라고 가정하고, 액터 신경망이 각각의 행동변수가 갖는 가우시안 분포의 평균값과 분산을 출력하게 한다.

### 그러면 아래 그림과 같을 것이고, 액터 신경망의 출력층의 뉴런 개수는 행동변수 차원당 2개(평균값과 분산)면 된다.

![6](https://user-images.githubusercontent.com/52277776/81943449-5b287580-9636-11ea-9158-6fe6001cec66.jpg)

### 기댓값이 $\mu=[\mu_1\mu_2...\mu_m]^T$ 이고 공분산이 $P=diag\{\sigma_{1}^2, \sigma_{2}^2, ..., \sigma_{m}^2\}$인 가우시안 정책 확률밀도함수일 때, 가우시안 로그-정책(log-policy) 확률밀도함수는 다음과 같이 나타낼 수 있다.

$$log\pi_\theta (u_t\mid x_t)=\sum_{j=1}^mlog\pi_\theta(u_{t,j}\mid x_t)=$$ <br>$$ -\sum_{j=1}^m[\frac{(u_{t,j}-\mu_{\theta,j}(x_t)^2)}{2\sigma_{\theta,j}^2(x_t)}+\frac{1}{2}log(2\pi\sigma_{\theta,j}^2(x_t))]$$

### 이를 keras 및 tensorflow를 이용하여 구현해 보자.